In [36]:
from netCDF4 import Dataset
from datetime import datetime, timedelta
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt 
import pickle
%matplotlib inline

info = []

#read the Data sites file (in different folder as this document)
f = open('../DataSites.csv','r')

# Reading each line of the file, skipping the first (non-data) line
for line in f:
    if line[0:2] == 'ID':
        continue
    line = line.split(',')
    veg_type = line[3]
    lat = float(line[4])
    lon = float(line[5])
    info.append([line[1],lat,lon,veg_type])
    
f.close()
# print(info)

In [37]:
# Creating dictionary
data = [{} for i in range(len(info))]
mdata = np.zeros((len(info), 168, 4))*np.nan

for year in range(2005,2019):
    path_s0 = '/g/data/oe9/project/to_hot_in_here/soil_moisture/s0_pct_'+ str('%i' % year) + '_Actual_day.nc'
    path_ss = '/g/data/oe9/project/to_hot_in_here/soil_moisture/ss_pct_'+ str('%i' % year) + '_Actual_day.nc'
    path_sd = '/g/data/oe9/project/to_hot_in_here/soil_moisture/sd_pct_'+ str('%i' % year) + '_Actual_day.nc'
    url_fmc = 'http://dapds00.nci.org.au/thredds/dodsC/ub8/au/FMC/c6/mosaics/fmc_c6_'+ str('%i' % year) + '.nc'
    s0 = Dataset(path_s0, 'r')
    ss = Dataset(path_ss, 'r')
    sd = Dataset(path_sd, 'r')
    fmc = Dataset(url_fmc, 'r')
    
    #Defining latitude and longitude from fuel moisture data
    lat_fmc = fmc['latitude'][:]
    lon_fmc = fmc['longitude'][:]
    
    #Defining latitude and longitude from soil moisture data (top layer same as bottom layer)
    lat_s0 = s0['latitude'][:]
    lon_s0 = s0['longitude'][:]
    
    #Matching timesteps between FMC and soil moisture
    #To match the times that we have converted, 
    #for example we have 92(every 4 days) values we need to match to a list with 365 values (daily)
    time = fmc['time'][:]
    for i in range (time.size):
        x = datetime(1970,1,1) + timedelta(seconds = int(time[i]))
        x = x-datetime(1900,1,1)
        time[i] = x.days
    time2=s0['time'][:]
    time3 =[]
    for i in time:
        j=np.where(time2 == int(i))[0][0]
        time3.append(j)
    time3 = np.array(time3)
    
    for site in range(len(info)):
        #Finding closest FMC cell
        a = np.where((lat_fmc < info[site][1]+0.00251) & (lat_fmc > info[site][1]-0.00251))[0][0]
        b = np.where((lon_fmc < info[site][2]+0.00251) & (lon_fmc > info[site][2]-0.00251))[0][0]
        #Finding closest SM cell
        c = np.where((lat_s0 < info[site][1]+0.0251) & (lat_s0 > info[site][1]-0.0251))[0][0]
        d = np.where((lon_s0 < info[site][2]+0.0251) & (lon_s0 > info[site][2]-0.0251))[0][0]
        
        for m in range(1,13):
            t0 = datetime(year, m, 1) - datetime(1900, 1, 1)
            t0 = t0.days
            tn = datetime(year, m, 1) + timedelta(days = 40) 
            tn = datetime(tn.year, tn.month, 1)- datetime(1900, 1, 1)
            tn = tn.days
            smw = np.where((time2 >= t0) & (time2 < tn))[0]
            fmcw = np.where((time >= t0) & (time < tn))[0]
            
            mi = (year - 2005)*12 + m - 1
            
            mdata[site, mi, 0] = np.nanmean(fmc['fmc_mean'][fmcw,a-1:a+2,b-1:b+2])
            mdata[site, mi, 1] = np.nanmean(s0['s0_pct'][smw,c,d])
            mdata[site, mi, 2] = np.nanmean(ss['ss_pct'][smw,c,d])
            mdata[site, mi, 3] = np.nanmean(sd['sd_pct'][smw,c,d])
            
        
        # making a list of 3 elements (fmc, s0, ss) for each time for each site
        for j, t in enumerate(time3):
            dse = time2[t]
            #v1 = np.mean(fmc['fmc_mean'][j,a,b])
            v1 = np.mean(fmc['fmc_mean'][j,a-1:a+2,b-1:b+2])
            v2 = float(s0['s0_pct'][t,c,d].data)
            v3 = float(ss['ss_pct'][t,c,d].data)
            v4 = float(sd['sd_pct'][t,c,d].data)
#             if j == 0:
#                 print('site, dse, v1, v2, v3 =', site, dse, v1, v2, v3)
            data[site][dse] = [v1, v2, v3, v4]
    print(year)
    
#save as pickle file
with open('maindata.pkl', 'wb') as f:
    pickle.dump([info, data, mdata], f)

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [38]:
# trying other fits
# copied from multiple_soilmoisture

def linear(x, a, b):
    return a * x + b

#def glf(x, A, K, C, Q, B, ni):
#    return A + (K - A) / ((C + Q * np.exp(-B * x)) ** (1./ni))

def glf(x, A, K, B, ni):
    return A + (K - A) / (np.exp(-B * x) ** (1./ni))

def exponencial(x, a, b, c):
    return a + b * np.exp(c*x)

def powerlaw(x, a, b, c):
    return a + b * (x**c)

def fit_best(xdata, ydata):
    """ Fits many function types to the provided x,y datasets and return the best based on r² value.
    The functions are:
        (1) linear regression;
        (2) generalised logistic function;
        (3) an exponencial function and;
        (4) power-law function
    """
    bestmodel = 1
    
    foundfit = False
    
    # Fitting the linear model first
    try:
        param_lin, pcov = curve_fit(linear, xdata, ydata, maxfev=10000)
        yf = linear(xdata, *param_lin)
        r, p = stats.pearsonr(ydata, yf)
        r2max = r ** 2
        foundfit = True
    except RuntimeError:
        r2max = 0
    
    
    # Fitting the generalised logistic function
    try:
        param_glf, pcov = curve_fit(glf, xdata, ydata, maxfev=10000)
        yf = glf(xdata, *param_glf)
        r, p = stats.pearsonr(ydata, yf)
        r2 = r ** 2
        foundfit = True

        if r2 > r2max:
            bestmodel = 2
            r2max = r2
            
    except RuntimeError:
        r2max = 0
    
    # Fitting the exponencial function
    try:
        param_exp, pcov = curve_fit(exponencial, xdata, ydata, maxfev=10000)
        yf = exponencial(xdata, *param_exp)
        r, p = stats.pearsonr(ydata, yf)
        r2 = r ** 2
        foundfit = True

        if r2 > r2max:
            bestmodel = 3
            r2max = r2
            
    except RuntimeError:
        r2max = 0
    
    # Fitting the power-law function
    try:
        param_law, pcov = curve_fit(powerlaw, xdata, ydata, maxfev=10000)
        yf = powerlaw(xdata, *param_law)
        r, p = stats.pearsonr(ydata, yf)
        r2 = r ** 2
        foundfit = True

        if r2 > r2max:
            bestmodel = 4
            r2max = r2
    
    except RuntimeError:
        r2max = 0

    if foundfit:
        
        # Use the best fit to create a sample of 100 x,y pairs of model data
        xi = np.linspace(np.min(xdata), np.max(xdata), 100)

        if bestmodel == 1:
            yi = linear(xi, *param_lin)
        elif bestmodel == 2:
            yi = glf(xi, *param_glf)
        elif bestmodel == 3:
            yi = exponencial(xi, *param_exp)
        else:
            yi = powerlaw(xi, *param_law)

        return xi, yi, r2max
    
    else:
        return xi, xi*np.nan, r2max

In [39]:
#with open('maindata.pkl', 'rb') as f:
#    info, data, mdata = pickle.load(f)

#Compute the linear R² for each site (before other regressions)
NS = range(len(info))
table = np.zeros((len(info), 6)) - 1

for si in NS:
    
    dt = sorted(data[si]) #list of datetimes sorted from oldest to newest
    fmclr = np.array([data[si][t][0] for t in dt]) #FMC
    s0lr = np.array([data[si][t][1] for t in dt]) #upper soil moisture
    sslr = np.array([data[si][t][2] for t in dt]) #lower soil moisture
    sdlr = np.array([data[si][t][3] for t in dt]) #deep soil moisture
    
    mask = np.isfinite([s0lr, fmclr]).all(axis=0)     #take only when both are non-Nan values   
    slope, intercept, r_value, p_value, std_err = stats.linregress(s0lr[mask], fmclr[mask])
    table[si,0] = r_value**2
    
    xm, ym, r2 = fit_best(s0lr[mask], fmclr[mask])
    table[si,1] = r2
    
    mask = np.isfinite([sslr, fmclr]).all(axis=0)            
    slope, intercept, r_value, p_value, std_err = stats.linregress(sslr[mask], fmclr[mask])
    table[si,2] = r_value**2
    
    xm, ym, r2 = fit_best(sslr[mask], fmclr[mask])
    table[si,3] = r2
    
    mask = np.isfinite([sdlr, fmclr]).all(axis=0)            
    slope, intercept, r_value, p_value, std_err = stats.linregress(sdlr[mask], fmclr[mask])
    table[si,4] = r_value**2
    
    xm, ym, r2 = fit_best(sdlr[mask], fmclr[mask])
    table[si,5] = r2

/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in power
/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in power
/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Warning: converting a masked element to nan.
  # This is added back by InteractiveShellApp.init_path()
/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in multiply
/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [40]:
tbmonth = np.zeros((len(info), 6)) - 1

for si in NS:
    
    fmclr = mdata[si,:,0] #FMC
    s0lr = mdata[si,:,1] #upper soil moisture
    sslr = mdata[si,:,2] #lower soil moisture
    sdlr = mdata[si,:,3] #deep soil moisture
    
    mask = np.isfinite([s0lr, fmclr]).all(axis=0)     #take only when both are non-Nan values   
    slope, intercept, r_value, p_value, std_err = stats.linregress(s0lr[mask], fmclr[mask])
    tbmonth[si,0] = r_value**2
    
    xm, ym, r2 = fit_best(s0lr[mask], fmclr[mask])
    tbmonth[si,1] = r2
                 
    mask = np.isfinite([sslr, fmclr]).all(axis=0)            
    slope, intercept, r_value, p_value, std_err = stats.linregress(sslr[mask], fmclr[mask])
    tbmonth[si,2] = r_value**2
    
    xm, ym, r2 = fit_best(sslr[mask], fmclr[mask])
    tbmonth[si,3] = r2
    
    mask = np.isfinite([sdlr, fmclr]).all(axis=0)            
    slope, intercept, r_value, p_value, std_err = stats.linregress(sdlr[mask], fmclr[mask])
    tbmonth[si,4] = r_value**2
    
    xm, ym, r2 = fit_best(sdlr[mask], fmclr[mask])
    tbmonth[si,5] = r2

/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in power
/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in multiply
/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in power
/g/data/oe9/software/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [56]:
# plotting r2 values for ss and s0 through the sites and sd

x = np.array(list(NS), dtype = int)
ig = np.array([i for i in NS if info[i][3] == 'Grassland'], dtype = int)
ish = np.array([i for i in NS if info[i][3] == 'Shrubland'], dtype = int)
ifo = np.array([i for i in NS if info[i][3] == 'Forest'], dtype = int)
names = ['Upper soil moisture', 'Lower soil moisture', 'Deep soil moisture']

for layer in range(3):

    fig = plt.figure(figsize=(14,6))
    ax11 = plt.subplot2grid((1, 5), (0, 0), colspan=2) #grasslands
    ax12 = plt.subplot2grid((1, 5), (0, 2)) #shrublands
    ax13 = plt.subplot2grid((1, 5), (0, 3), colspan=2) #forest
    # ax21 =  plt.subplot2grid((2, 6), (1, 0), colspan=3) #grasslands
    # ax22 = plt.subplot2grid((2, 6), (1, 3)) #shrublands
    # ax23 = plt.subplot2grid((2, 6), (1, 4), colspan=2) #forest
    
    ax11.plot(x[ig], table[ig,2*layer+1], label = 'Best 4-Daily')
    ax12.plot(x[ish], table[ish,2*layer+1], label = 'Best 4-Daily')
    ax13.plot(x[ifo], table[ifo,2*layer+1], label = 'Best 4-Daily')
#     ax21.plot(x[ig], table[ig,3], label = 'Best 4-Daily')
#     ax22.plot(x[ish], table[ish,3], label = 'Best 4-Daily')
#     ax23.plot(x[ifo], table[ifo,3], label = 'Best 4-Daily')

    ax11.plot(x[ig], tbmonth[ig,2*layer+1], label = 'Best Monthly')
    ax12.plot(x[ish], tbmonth[ish,2*layer+1], label = 'Best Monthly')
    ax13.plot(x[ifo], tbmonth[ifo,2*layer+1], label = 'Best Monthly')
#     ax21.plot(x[ig], tbmonth[ig,3], label = 'Best Monthly')
#     ax22.plot(x[ish], tbmonth[ish,3], label = 'Best Monthly')
#     ax23.plot(x[ifo], tbmonth[ifo,3], label = 'Best Monthly')

    ax11.plot(x[ig], table[ig,2*layer], label = '4-Daily')
    ax12.plot(x[ish], table[ish,2*layer], label = '4-Daily')
    ax13.plot(x[ifo], table[ifo,2*layer], label = '4-Daily')
#     ax21.plot(x[ig], table[ig,1], label = '4-Daily')
#     ax22.plot(x[ish], table[ish,1], label = '4-Daily')
#     ax23.plot(x[ifo], table[ifo,1], label = '4-Daily')

    ax11.plot(x[ig], tbmonth[ig,2*layer], label = 'Monthly')
    ax12.plot(x[ish], tbmonth[ish,2*layer], label = 'Monthly')
    ax13.plot(x[ifo], tbmonth[ifo,2*layer], label = 'Monthly')
#     ax21.plot(x[ig], tbmonth[ig,1], label = 'Monthly')
#     ax22.plot(x[ish], tbmonth[ish,1], label = 'Monthly')
#     ax23.plot(x[ifo], tbmonth[ifo,1], label = 'Monthly')

    ax11.set_ylabel('R$^2$')
    ax11.set_yticks([o*0.1 for o in range(11)])
    ax12.set_yticks([o*0.1 for o in range(11)])
    ax13.set_yticks([o*0.1 for o in range(11)])
    ax13.set_xticks(list(range(16,31,2)))
    #ax21.set_ylabel('R$^2$')

    for j, ax in enumerate([ax11, ax12, ax13]):
        ax.set_ylim(0, 1)
        #ax.get_yaxis().set_ticks([])
        #ax.set_xticks([0,2,4,6,8,10])
        #ax.set_title(['Upper soil moisture', 'Lower soil moisture'][j])
        ax.legend()
        ax.grid()
        ax.set_xlabel('Site no.')
        if j != 0:
            ax.set_yticklabels([])
        if j == 0:
            ax.set_title('Grassland\n'+names[layer])
            ax.plot([x[ig][0], x[ig][-1]], [0.5, 0.5], 'k-')
        elif j == 1:
            ax.set_title('Shrubland\n'+names[layer])
            ax.plot([x[ish][0], x[ish][-1]], [0.5, 0.5], 'k-')
        elif j == 2:
            ax.set_title('Forest\n'+names[layer])
            ax.plot([x[ifo][0], x[ifo][-1]], [0.5, 0.5], 'k-')
        

#fig, axes = plt.subplots(nrows=2,ncols=1, figsize=(12,8), squeeze = False)
#for j, ax in enumerate(axes.flatten()):
    
#     ax.plot(x, table[:,j], label = names[j])
    
#     ax.set_ylim(0, 1)
#     #ax.set_xticks([0,2,4,6,8,10])
#     ax.plot([x[0], x[-1]], [0.5, 0.5], 'k-')
#     ax.set_title(['Upper soil moisture', 'Lower soil moisture'][j])
#     ax.legend()
#     ax.grid()
#     ax.set_xlabel('Site no.')
#     ax.set_ylabel('R$^2$')

    txt = ''
    #print(mdata['Unnamed: 0'].tolist())

    for i in NS:
        txt += str('%2.2i: %s\n' %(i, info[i][0]))

    fig.text(0.78, 0.05, txt, transform=fig.transFigure, size=11)

    plt.tight_layout()
    plt.subplots_adjust(right = 0.77)

    #plt.show()
    #plt.close(fig)

    plt.savefig('r2plot_layer%1i.png' % layer)
    plt.close(fig)
